In [20]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize, sent_tokenize
import os

In [3]:
paragraphs = pd.read_csv("data/share_repurchase_paragraphs.csv")

In [4]:
paragraphs.head()

,ticker,accession_number,data_key_friendly_name,text,data_value,reported_data_value,reported_units,paragraph_text
0,A,0001090872-17-000018,Share Repurchase Authorization Date,"May 28, 2015",20150528,20200000.00,ones,"On May 28, 2015 we announced that our board ..."
1,A,0001090872-17-000018,Share Repurchase Authorization,The 2015 share repurchase program authorizes t...,1140000000,1.14,billions,"On May 28, 2015 we announced that our board ..."
2,A,0001090872-17-000018,Share Repurchase Intention,remaining authorization to repurchase up to,610000000,610.00,millions,"Table of Contents 2016, upon the completion ..."
3,A,0001090872-17-000018,Share Repurchase Count,repurchased,4100000,4.10,millions,"Table of Contents 2016, upon the completion ..."
4,A,0001090872-17-000018,Amount Spent on Share Repurchases,repurchased,194000000,194.00,millions,"Table of Contents 2016, upon the completion ..."


In [6]:
paragraphs.paragraph_text[0]

'On May 28, 2015 we  announced that our board  of directors had approved a  new share repurchase program  (the   "2015 repurchase program"). The 2015 share repurchase program authorizes the purchase of up to $1.14  billion   of our common stock at the company\'s discretion  through and including November 1, 2018. The 2015  repurchase   program does  not require  the company  to  acquire a  specific number  of  shares and  may be  suspended  or   discontinued at any time. During the year ended October 31,                                                        95'

In [14]:
test_p = paragraphs.paragraph_text[2]

In [19]:
test_p

'Table of Contents   2016, upon the completion of our previous repurchase program, we repurchased approximately 8.3 million shares   for $336  million  under  this  authorization.  During  the  year  ended  October  31,  2017  we  repurchased   approximately 4.1 million shares for $194  million under this authorization. As  of October 31, 2017, we  had   remaining authorization to repurchase up to $610 million of our common stock under this program'

In [9]:
paragraphs.text[1]

'The 2015 share repurchase program authorizes the purchase of up to'

for root, dirs, files in os.walk("data/nc_training_filings", topdown=True):
    for file in files:
        print(file)
        # df = pd.read_csv(os.path.join(root, file), encoding = "ISO-8859-1", low_memory = False)
        # nih_full = nih_full.append(df)

In [21]:
test_p_sent = sent_tokenize(test_p)

In [22]:
test_p_sent

['Table of Contents   2016, upon the completion of our previous repurchase program, we repurchased approximately 8.3 million shares   for $336  million  under  this  authorization.',
 'During  the  year  ended  October  31,  2017  we  repurchased   approximately 4.1 million shares for $194  million under this authorization.',
 'As  of October 31, 2017, we  had   remaining authorization to repurchase up to $610 million of our common stock under this program']

In [26]:
test_p_sent_bow = list()
for sent in test_p_sent:
    test_p_sent_bow.append(word_tokenize(sent))

In [27]:
test_p_sent_bow

[['Table',
  'of',
  'Contents',
  '2016',
  ',',
  'upon',
  'the',
  'completion',
  'of',
  'our',
  'previous',
  'repurchase',
  'program',
  ',',
  'we',
  'repurchased',
  'approximately',
  '8.3',
  'million',
  'shares',
  'for',
  '$',
  '336',
  'million',
  'under',
  'this',
  'authorization',
  '.'],
 ['During',
  'the',
  'year',
  'ended',
  'October',
  '31',
  ',',
  '2017',
  'we',
  'repurchased',
  'approximately',
  '4.1',
  'million',
  'shares',
  'for',
  '$',
  '194',
  'million',
  'under',
  'this',
  'authorization',
  '.'],
 ['As',
  'of',
  'October',
  '31',
  ',',
  '2017',
  ',',
  'we',
  'had',
  'remaining',
  'authorization',
  'to',
  'repurchase',
  'up',
  'to',
  '$',
  '610',
  'million',
  'of',
  'our',
  'common',
  'stock',
  'under',
  'this',
  'program']]

In [48]:
possible_important_words = ["remaining"]

In [57]:
months = ["January", "Febuary", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

In [66]:
units = ["hundred", "thousand", "million", "billion", "trillion"]

In [67]:
# This is how to check to see if word in list is in another list
# any(word in test_p_sent_bow[2] for word in possible_important_words)

In [68]:
for sent in test_p_sent_bow:
    if any(word in sent for word in possible_important_words):
        print(set(possible_important_words) & set(sent))
        for ind, token in enumerate(sent):
            if token in months:
                print("month: ", sent[ind])
            elif token == "$":
                print("amount: ", sent[ind+1])#, sent[ind+2])
            elif token in units:
                print("units: ", sent[ind])

{'remaining'}
month:  October
amount:  610
units:  million
